In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from collections import Counter

import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')
nltk.download('punkt')
from nltk.tokenize import word_tokenize
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

#import os
#for dirname, _, filenames in os.walk('/kaggle/input'):
#    for filename in filenames:
#        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [ ]:
#data = pd.read_csv("/kaggle/input/hate-speech-and-offensive-language-dataset/labeled_data.csv")

from google.colab import drive
from google.colab import files
drive.mount('/content/drive/')

data = pd.read_csv("/content/drive/MyDrive/ACMResearch/changedData.csv")

df = pd.DataFrame(data) 

df.head()

Mounted at /content/drive/


,Unnamed: 0,count,hate_speech,offensive_language,neither,class,tweet
0,0,3,0,0,3,2,!!! RT @mayasolovely: As a woman you shouldn't...
1,1,3,0,3,0,0,!!!!! RT @mleew17: boy dats cold...tyga dwn ba...
2,2,3,0,3,0,0,!!!!!!! RT @UrKindOfBrand Dawg!!!! RT @80sbaby...
3,3,3,0,2,1,0,!!!!!!!!! RT @C_G_Anderson: @viva_based she lo...
4,4,6,0,6,0,0,!!!!!!!!!!!!! RT @ShenikaRoberts: The shit you...


In [ ]:
df.isnull().sum()

Unnamed: 0            0
count                 0
hate_speech           0
offensive_language    0
neither               0
class                 0
tweet                 0
dtype: int64

In [ ]:
class_tweet = df[['class', 'tweet']]
class_tweet.head()

,class,tweet
0,2,!!! RT @mayasolovely: As a woman you shouldn't...
1,0,!!!!! RT @mleew17: boy dats cold...tyga dwn ba...
2,0,!!!!!!! RT @UrKindOfBrand Dawg!!!! RT @80sbaby...
3,0,!!!!!!!!! RT @C_G_Anderson: @viva_based she lo...
4,0,!!!!!!!!!!!!! RT @ShenikaRoberts: The shit you...


In [ ]:
def remove_tags(text):
  text = text[text.find(":") + 1:]
  return text

In [ ]:
def convert_to_one(classification):
  if(classification == 2):
    classification = 1
  return classification

In [ ]:
class_tweet['tweet'] = class_tweet['tweet'].apply(remove_tags)
class_tweet['class'] = class_tweet['class'].apply(convert_to_one)

<ipython-input-9-dad1383e86a5>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  class_tweet['tweet'] = class_tweet['tweet'].apply(remove_tags)
<ipython-input-9-dad1383e86a5>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  class_tweet['class'] = class_tweet['class'].apply(convert_to_one)


In [ ]:
class_tweet['class'].value_counts()

0    20620
1     4163
Name: class, dtype: int64

In [ ]:
import imblearn
from imblearn.under_sampling import RandomUnderSampler

#Random under-sampling
randomUnder = RandomUnderSampler(random_state=42, replacement=True, sampling_strategy='majority')
X_resampled, y_resampled = randomUnder.fit_resample(class_tweet['tweet'].values.reshape(-1,1), class_tweet['class'])
print('Original dataset shape: ', Counter(class_tweet['class']))
print('Resampled dataset shape: ',Counter(y_resampled))

Original dataset shape:  Counter({0: 20620, 1: 4163})
Resampled dataset shape:  Counter({0: 4163, 1: 4163})


In [ ]:
from sklearn.model_selection import train_test_split  

x_tweet = pd.DataFrame(X_resampled, columns=['tweet'])
y_class = pd.DataFrame(y_resampled, columns=['class'])

x_tweet_train, x_tweet_test, y_class_train, y_class_test = train_test_split(x_tweet, y_class, test_size=0.25, random_state=0)

#bert_preprocess = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3") 
#bert_encoder = hub.KerasLayer('https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/4')

In [ ]:
x_tweet_train

,tweet
6795,I am a strong believer that birds of a feathe...
3376,"@Godbody_ream ""I want to eat your pussy"" &#128..."
729,"Yeah, #Eagles definitely aren't finishing 4th..."
7384,Oreo Cheesecake Bars http://t.co/RT27crIqZj
4536,@A_Little_Wicked that's not ghetto. That's swe...
...,...
4373,What will the Lakers be after kobe&#8221; trash
7891,Thinking bout those oysters at the gator bait ...
4859,@Nazario1017 I feel that.. i see nothing wrong...
3264,@Cryystalina haha yes niggah but yesterday nig...


In [ ]:
x_tweet_test

,tweet
6633,Part of my cousin Danny's Yankee Swap. Pretty...
7816,The Guy got excited when he saw Oreo cookie cr...
5412,@treagle05 early bird gets the worm tomcat
6949,Rick Scott vs. Charlie Crist is the worst gub...
7097,"""I&#8217;ve had the greatest job in the world..."
...,...
5810,Going to California today for some analyzer tr...
7251,You may cry after watching the Derek Jeter @G...
443,//t.co/djuvhsBy84
5770,Four ho&#251;rs to back up this iPhone haha......


In [ ]:
y_class_train

,class
6795,1
3376,0
729,0
7384,1
4536,1
...,...
4373,1
7891,1
4859,1
3264,0


In [ ]:
y_class_test

,class
6633,1
7816,1
5412,1
6949,1
7097,1
...,...
5810,1
7251,1
443,0
5770,1


In [ ]:
x_tweet_train.shape

(6244, 1)

In [ ]:
x_tweet_test.shape

(2082, 1)

In [ ]:
##no
#x_tweet_test_copy = x_tweet_test.copy()
#x_tweet_test_copy.reset_index(inplace = True)
#x_tweet_test_copy.shape

In [ ]:
##no
#x_tweet_train_copy = x_tweet_train.copy()
#x_tweet_train_copy.reset_index(inplace = True)
#x_tweet_train_copy.shape

In [ ]:
x_tweet_train = x_tweet_train.squeeze()
type(x_tweet_train.squeeze())

pandas.core.series.Series

In [ ]:
frames = [x_tweet_test, y_class_test]
result_concat = pd.concat(frames, axis=1, join="inner")
print(result_concat)

                                                  tweet  class
6633   Part of my cousin Danny's Yankee Swap. Pretty...      1
7816  The Guy got excited when he saw Oreo cookie cr...      1
5412         @treagle05 early bird gets the worm tomcat      1
6949   Rick Scott vs. Charlie Crist is the worst gub...      1
7097   "I&#8217;ve had the greatest job in the world...      1
...                                                 ...    ...
5810  Going to California today for some analyzer tr...      1
7251   You may cry after watching the Derek Jeter @G...      1
443                                   //t.co/djuvhsBy84      0
5770  Four ho&#251;rs to back up this iPhone haha......      1
84     " now i know why niggas have sides, my bitch ...      0

[2082 rows x 2 columns]


In [ ]:
just_hate = result_concat.loc[result_concat['class'] == 0]
x_tweet_test_hate = just_hate['tweet']
y_class_test_hate = just_hate['class']

print(x_tweet_test_hate)
print(y_class_test_hate)

916      Ok, sis." She'd rather be a broke bitch? *shr...
1768    Smashed a werewolf chick and thought she was r...
3726     Either Young Thug gay, or this bitch pooted h...
1066    @myfriendbriana can I ask my bitch for other b...
2521     That's because they're possibly tucking a pen...
                              ...                        
2329    @DefendWallSt those little #ISIS faggots usual...
1173     http://t.co/s0kAVO4D3E them Wet &#128076;&#12...
1027    @Jessycaa_Rubi @O_ShitItsLaura Bitch you just ...
443                                     //t.co/djuvhsBy84
84       " now i know why niggas have sides, my bitch ...
Name: tweet, Length: 1036, dtype: object
916     0
1768    0
3726    0
1066    0
2521    0
       ..
2329    0
1173    0
1027    0
443     0
84      0
Name: class, Length: 1036, dtype: int64


In [ ]:
x_tweet_test_hate = x_tweet_test_hate.squeeze()
type(x_tweet_test_hate)

pandas.core.series.Series

In [ ]:
y_class_test_hate = y_class_test_hate.squeeze()
type(y_class_test_hate)

pandas.core.series.Series

In [ ]:
x_tweet_test = x_tweet_test.squeeze()
type(x_tweet_test)

pandas.core.series.Series

In [ ]:
x_tweet_test.shape

(2082,)

In [ ]:
x_tweet_train.shape

(6244,)

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf_vectorizer = TfidfVectorizer(stop_words = stopwords.words('english'))
tfidf_train = tfidf_vectorizer.fit_transform(x_tweet_train)
tfidf_test = tfidf_vectorizer.transform(x_tweet_test)
tfidf_test.shape

(2082, 13246)

In [ ]:
tfidf_test_hate = tfidf_vectorizer.transform(x_tweet_test_hate)

In [ ]:
tfidf_test.toarray()

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [ ]:
#lists all the unique words it vectorized
#tfidf_vectorizer.vocabulary_

In [ ]:
y_class_test = y_class_test.squeeze()
type(y_class_test)

pandas.core.series.Series

In [ ]:
y_class_train = y_class_train.squeeze()
type(y_class_train)

pandas.core.series.Series

In [ ]:
#import pretrained model, import accuracy finder package (accuracy_score)
from sklearn.metrics import accuracy_score
#this pretrained model uses linear, while other was naive bayes
from sklearn.linear_model import PassiveAggressiveClassifier

#Finding best parameters using Grid Search
#testing following hyper parameter values:
#C=1.0, validation_fraction=0.1, class_weight=None
estimator = PassiveAggressiveClassifier()

hparams = {"C": [0.1, 1, 10, 100],
           "verbose": [0,1,2],
           "class_weight": ["balanced"]}
###from sklearn.model_selection import GridSearchCV
###GS_estimator = GridSearchCV(estimator, hparams, cv=5, scoring="accuracy")
###GS_estimator.fit(tfidf_train, y_class_train)
###print(GS_estimator.best_params_)



In [ ]:
from sklearn.metrics import accuracy_score
from sklearn.linear_model import PassiveAggressiveClassifier
#instantiate instance of own pretrained model
pac = PassiveAggressiveClassifier(C=0.1,class_weight='balanced')



#trains her pretrained model adapted to her own dataset using transfer learning
pac.fit(tfidf_train, y_class_train)

y_pred = pac.predict(tfidf_test) #Test or something
score = accuracy_score(y_class_test, y_pred)
print(f'Accuracy: {round(score*100,2)}%')

Accuracy: 93.52%


In [ ]:
y_pred_hate = pac.predict(tfidf_test_hate) #Test or something
score = accuracy_score(y_class_test_hate, y_pred_hate)
print(f'Accuracy: {round(score*100,2)}%')

Accuracy: 91.12%


In [ ]:
print(tfidf_test_hate)
print(y_class_test_hate)
print(y_pred_hate)

  (0, 12865)	0.47036385222761806
  (0, 11402)	0.3125090482788142
  (0, 10468)	0.45779883141008276
  (0, 9424)	0.40399480647880753
  (0, 8334)	0.3743709582296751
  (0, 2046)	0.37973779220857
  (0, 1714)	0.15475775380230353
  (1, 12655)	0.339774675542785
  (1, 11581)	0.22283369216424995
  (1, 10599)	0.3245457513979161
  (1, 9563)	0.3137406577619968
  (1, 6758)	0.2724777158363398
  (1, 6631)	0.17291247794735368
  (1, 5513)	0.2114077720242768
  (1, 5189)	0.3137406577619968
  (1, 4384)	0.29272196536574235
  (1, 3699)	0.339774675542785
  (1, 3592)	0.339774675542785
  (1, 2533)	0.25939561303615094
  (2, 13128)	0.39999920994819776
  (2, 11608)	0.4904935568037989
  (2, 5648)	0.37657917496066085
  (2, 4857)	0.39999920994819776
  (2, 3979)	0.4193564481500972
  (2, 2751)	0.1761981400604953
  :	:
  (1032, 5065)	0.1862760523530421
  (1032, 2751)	0.12487443067625763
  (1032, 563)	0.17637062789609714
  (1032, 157)	0.6422187579867294
  (1032, 118)	0.2928271403039847
  (1033, 11282)	0.2754657445149092
 

In [ ]:
resampled_df = pd.concat([x_tweet, y_class], axis=1)
resampled_df.head()

,tweet,class
0,When you listening to a bitch complaining abo...,0
1,Bae: go text your hoes \nMe: https://t.co/5PA...,0
2,@kingjvsh @_hanimA Don't be a bitch,0
3,You lil stupid ass bitch i aint fuckin wit you,0
4,Oh I'm nasty for smoking cigarettes? And you'r...,0


In [ ]:
hate_data_eda = pd.read_csv("/content/drive/MyDrive/ACMResearch/augmented_hate_tweets.csv")
hate_df = pd.DataFrame(hate_data_eda) 
normal_df = resampled_df[resampled_df['class'] == 1]
df_combine = pd.concat([hate_df, normal_df])
#df_combine = df_combine.dropna()
df_combine_class = df_combine['class']
df_combine_tweet = df_combine['tweet']
df_combine.head()

,Unnamed: 0,Tweets,0,tweet,class
0,0.0,NaN,"""@Blackman38Tide: @WhaleLookyHere @HowdyDowdy1...",NaN,NaN
1,0.0,NaN,"""@CB_Baby24: @white_thunduh alsarabsss"" hes a ...",NaN,NaN
2,0.0,NaN,"""@DevilGrimz: @VigxRArts you're blooming jovia...",NaN,NaN
3,0.0,NaN,"""@MarkRoundtreeJr: LMFAOOOO why HATE BLACK PEO...",NaN,NaN
4,0.0,NaN,"""@NoChillPaz: ""At least I'm RGJa7CfoiT a nigge...",NaN,NaN


In [ ]:
#eda
hate_data_eda = pd.read_csv("/content/drive/MyDrive/ACMResearch/augmented_hate_tweets.csv")
hate_df = pd.DataFrame(hate_data_eda) 
normal_df = resampled_df[resampled_df['class'] == 1]
df_combine = pd.concat([hate_df, normal_df])
df_combine = df_combine.dropna()
df_combine_class = df_combine['class']
df_combine_tweet = df_combine['tweet']

#hate_tweet = hate_df[['0']] #takes all hate tweets from augmented_hate_tweets.csv
#hate_class =hate_df.iloc[:, 0] #takes all the class numbers from csv file and puts them in here

#hate_tweet = hate_tweet.squeeze()
#hate_class = hate_class.squeeze()

df_combine_tweet = df_combine_tweet.squeeze()
df_combine_class = df_combine_class.squeeze()

tfidf_test_hate = tfidf_vectorizer.transform(df_combine_tweet)
y_pred = pac.predict(tfidf_test_hate)

score = accuracy_score(df_combine_class, y_pred)
print(f'% of Predicted Hate still being Predicted as Hate (PAC): {round(score*100,2)}%')

#word embedding
hate_data_glove = pd.read_csv("/content/drive/MyDrive/ACMResearch/word_embedding_glove.csv")
hate_df_2 = pd.DataFrame(hate_data_glove)
df_combine = pd.concat([hate_df_2, normal_df])
df_combine_class = df_combine['class']
df_combine_tweet = df_combine['tweet']

hate_tweet_2 = hate_df_2[['tweet']]
#hate_class_2 = hate_df_2.iloc[:, ]

#hate_tweet_2 = hate_tweet_2.squeeze()
#hate_class_2 = hate_class_2.squeeze()

df_combine_tweet = df_combine_tweet.squeeze()
df_combine_class = df_combine_class.squeeze()

tfidf_test_hate_2 = tfidf_vectorizer.transform(df_combine_tweet)
y_pred_2 = pac.predict(tfidf_test_hate_2)

score = accuracy_score(df_combine_class, y_pred_2)
print(f'% of Predicted Hate still being Predicted as Hate (PAC): {round(score*100,2)}%')


ValueError: ignored

In [ ]:
import matplotlib.pyplot as plt
import itertools
def plot_confusion_matrix(cm, classes,
                          normalize=False,
                          title='Confusion matrix',
                              cmap=plt.cm.Blues):

    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)

    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]

    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, cm[i, j],
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")

    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')

In [ ]:
from sklearn.metrics import confusion_matrix
CM = confusion_matrix(y_class_test, y_pred)
plot_confusion_matrix(CM, classes=['hate', 'not hate']) #wow

NameError: ignored

In [ ]:
from sklearn.ensemble import RandomForestClassifier

estimator = RandomForestClassifier()

hparams = {"n_estimators": [100],
           "criterion": ["gini", "entropy", "log_loss"],
           "max_features": ["sqrt", "log2"]}
###from sklearn.model_selection import GridSearchCV
###GS_estimator = GridSearchCV(estimator, hparams, cv=5, scoring="accuracy")
###GS_estimator.fit(tfidf_train, y_class_train)
###print(GS_estimator.best_params_)

In [ ]:
from sklearn.ensemble import RandomForestClassifier
#instantiate instance of own pretrained model
rfc = RandomForestClassifier(criterion="log_loss", max_features="sqrt")

#trains her pretrained model adapted to her own dataset using transfer learning
rfc.fit(tfidf_train, y_class_train)

y_pred_rfc = rfc.predict(tfidf_test) #Test or something
score_rfc = accuracy_score(y_class_test,y_pred_rfc)
print(f'Accuracy: {round(score_rfc*100,2)}%')

Accuracy: 90.78%


In [ ]:
y_pred_rfc_hate = rfc.predict(tfidf_test_hate) #Test or something
score = accuracy_score(y_class_test_hate, y_pred_rfc_hate)
print(f'Accuracy: {round(score*100,2)}%')

Accuracy: 93.73%


In [ ]:
#eda
y_pred = rfc.predict(tfidf_test_hate)

score = accuracy_score(hate_class,y_pred)
print(f'% of Predicted Hate still being Predicted as Hate (RFC): {round(score*100,2)}%')

#word embedding
y_pred_2 = rfc.predict(tfidf_test_hate_2)

score = accuracy_score(hate_class, y_pred_2)
print(f'% of Predicted Hate still being Predicted as Hate (RFC): {round(score*100,2)}%')

In [ ]:
from sklearn.metrics import confusion_matrix
CM = confusion_matrix(y_class_test, y_pred_rfc)
plot_confusion_matrix(CM, classes=['hate', 'not hate']) #wow

NameError: ignored

In [ ]:
from sklearn.neural_network import MLPClassifier
estimator = MLPClassifier()

hparams = {"hidden_layer_sizes": [64, 100],
           "activation": ["logistic", "tanh", "relu"],
           "learning_rate": ["constant", "invscaling", "adaptive"]}
###from sklearn.model_selection import GridSearchCV
###GS_estimator = GridSearchCV(estimator, hparams, cv=5, scoring="accuracy")
###GS_estimator.fit(tfidf_train, y_class_train)
###print(GS_estimator.best_params_)

In [ ]:
mlp = MLPClassifier(hidden_layer_sizes=100, activation='logistic', learning_rate='adaptive')

mlp.fit(tfidf_train, y_class_train)

y_pred_mlp = mlp.predict(tfidf_test)
score_mlp = accuracy_score(y_class_test,y_pred_mlp)
print(f'Accuracy: {round(score_mlp*100,2)}%')

Accuracy: 91.4%


In [ ]:
y_pred_mlp_hate = mlp.predict(tfidf_test_hate)
score_mlp = accuracy_score(y_class_test_hate, y_pred_mlp_hate)
print(f'Accuracy: {round(score_mlp*100,2)}%')

Accuracy: 90.73%


In [ ]:
#eda
y_pred = mlp.predict(tfidf_test_hate)

score = accuracy_score(hate_class,y_pred)
print(f'% of Predicted Hate still being Predicted as Hate (MLP): {round(score*100,2)}%')

#word embedding
y_pred_2 = mlp.predict(tfidf_test_hate_2)

score = accuracy_score(hate_class, y_pred_2)
print(f'% of Predicted Hate still being Predicted as Hate (MLP): {round(score*100,2)}%')

In [ ]:
from sklearn.metrics import confusion_matrix
CM = confusion_matrix(y_class_test, y_pred_mlp)
plot_confusion_matrix(CM, classes=['hate', 'not hate']) #wow

In [ ]:
test_input = ['you suck']
tfidf_test_input = tfidf_vectorizer.transform(test_input)

test_prediction1 = pac.predict(tfidf_test_input)
print("pac: ", test_prediction1)

if test_prediction1[0] == 0:
    print("pac: hate")
elif test_prediction1[0] == 1:
    print("pac: offensive")
else:
    print("pac: nah")
    

test2_prediction1 = mnb.predict(tfidf_test_input)
print("mnb: ", test2_prediction1)

if test2_prediction1[0] == 0:
    print("mnb: hate")
elif test2_prediction1[0] == 1:
    print("mnb: offensive")
else:
    print("mnb: nah")

In [ ]:
test_input = ['you big idiot know nothing useful u asshole']


tfidf_test_input = tfidf_vectorizer.transform(test_input)

test_prediction2 = pac.predict(tfidf_test_input)
print("pac: ", test_prediction2)

if test_prediction2[0] == 0:
    print("pac: hate")
elif test_prediction2[0] == 1:
    print("pac: offensive")
else:
    print("pac: nah")
    
    
test2_prediction2 = mnb.predict(tfidf_test_input)
print("mnb: ", test2_prediction2)

if test2_prediction2[0] == 0:
    print("mnb: hate")
elif test2_prediction2[0] == 1:
    print("mnb: offensive")
else:
    print("mnb: nah")

In [ ]:
test_input = ['you']


tfidf_test_input = tfidf_vectorizer.transform(test_input)

test_prediction3 = pac.predict(tfidf_test_input)
print("pac: ", test_prediction3)

if test_prediction3[0] == 0:
    print("pac: hate")
elif test_prediction3[0] == 1:
    print("pac: offensive")
else:
    print("pac: nah")
    
    
test2_prediction3 = mnb.predict(tfidf_test_input)
print("mnb: ", test2_prediction3)

if test2_prediction3[0] == 0:
    print("mnb: hate")
elif test2_prediction3[0] == 1:
    print("mnb: offensive")
else:
    print("mnb: nah")

In [ ]:
test_input = ['you are nice']


tfidf_test_input = tfidf_vectorizer.transform(test_input)

test_prediction4 = pac.predict(tfidf_test_input)
print("pac: ", test_prediction4)

if test_prediction4[0] == 0:
    print("pac: hate")
elif test_prediction4[0] == 1:
    print("pac: offensive")
else:
    print("pac: nah")
    
test2_prediction4 = mnb.predict(tfidf_test_input)
print("mnb: ", test2_prediction4)

if test2_prediction4[0] == 0:
    print("mnb: hate")
elif test2_prediction4[0] == 1:
    print("mnb: offensive")
else:
    print("mnb: nah")

In [ ]:
test_input = ["you bitch"]


tfidf_test_input = tfidf_vectorizer.transform(test_input)

test_prediction5 = pac.predict(tfidf_test_input)
print("pac: ", test_prediction5)

if test_prediction5[0] == 0:
    print("pac: hate")
elif test_prediction5[0] == 1:
    print("pac: offensive")
else:
    print("pac: nah")
    
    
test2_prediction5 = mnb.predict(tfidf_test_input)
print("mnb: ", test2_prediction5)

if test2_prediction5[0] == 0:
    print("mnb: hate")
elif test2_prediction5[0] == 1:
    print("mnb: offensive")
else:
    print("mnb: nah")

In [ ]:
test_input = ['you have an ugly face']


tfidf_test_input = tfidf_vectorizer.transform(test_input)

test_prediction6 = pac.predict(tfidf_test_input)
print("pac: ", test_prediction6)

if test_prediction6[0] == 0:
    print("pac: hate")
elif test_prediction6[0] == 1:
    print("pac: offensive")
else:
    print("pac: nah")
    
    
test2_prediction6 = mnb.predict(tfidf_test_input)
print("mnb: ", test2_prediction6)

if test2_prediction6[0] == 0:
    print("mnb: hate")
elif test2_prediction6[0] == 1:
    print("mnb: offensive")
else:
    print("mnb: nah")

In [ ]:
##past attempts
#from sklearn.tree import DecisionTreeClassifier

#clf = DecisionTreeClassifier()
#clf.fit(x_tweet_train, y_class_train)
#Accuracy = clf.score(x_tweet_test, y_class_test)
#print(Accuracy*100)